In [1]:
import sys
sys.path.insert(0, 'D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\Classes')

import warnings
warnings.filterwarnings('ignore')

from models import ClassificationModel, EmbeddingsModel, SiameseModel
from trainer import SiameseTrainerCombinationDataset, all_subjects
import torch

import wandb
!wandb login

wandb: Currently logged in as: svenk. Use `wandb login --relogin` to force relogin


In [2]:
subjects_test = [all_subjects[0]]
subjects_train = [sub for sub in all_subjects if sub not in subjects_test]

In [3]:
sweep_config = {
    'method': 'random',
    "early_terminate": {
          "type": "hyperband",
          "max_iter": 10,
          "s": 2
    },
    "metric": {
        'name': 'accuracy',
        'goal': 'maximize'
    },
    "parameters": {
        "lr": {'max': 0.01, 'min': 0.0001},
        "dropout": {'max': 0.5, 'min': 0.2},
        "last_layer": {"values": [64, 32, 16]}
    }
}

In [4]:
def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        
        trainer_hyperparameter = {
            "path": "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\FeatureGeneration\dataset_processed\XITE\\normalized_subjects.pkl",
            "subjects_train": subjects_train,
            "subjects_test": subjects_test,
            "wandb": True,
            "log": False,
            "learning_rate": config.lr,
            "batch_size": 256,
            "batch_size_test": 256,
            "freeze_embed": False,
            "decision_function": 0,
            "filter": [-1, -2, -3, -4, -5, -6],
            "weight_decay": None,
            "number_steps": 400
        }

        model_embedding_hyperparameter = {
            "sigmoid": False,
            "dropout": config.dropout,
            "layers": [207, 512, 64]
        }

        model_classifier_hyperparameter = {
            "dropout": config.dropout,
            "layers": [64, config.last_layer]
        }

        classifier_model = ClassificationModel(model_classifier_hyperparameter)
        embedding_model = EmbeddingsModel(model_embedding_hyperparameter)
        _ = embedding_model.load_state_dict(torch.load("D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\Results\models\embedding_models\model_abs_acc_89.40.pth"))
        siamese_model = SiameseModel(embedding_model, classifier_model, decision_function=trainer_hyperparameter["decision_function"])

        #filter = lambda data: data["label"].isin(trainer_hyperparameter["filter"])
        trainer = SiameseTrainerCombinationDataset(trainer_hyperparameter, siamese_model, device="cuda:0", filter=None)

        trainer.trainloop(10)

In [5]:
sweep_id = wandb.sweep(sweep_config, project="PainLevelShiftDetection_Combination_Dataset_SiameseNetwork")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: uxnlv2ji
Sweep URL: https://wandb.ai/svenk/PainLevelShiftDetection_Combination_Dataset_SiameseNetwork/sweeps/uxnlv2ji


In [6]:
wandb.agent(sweep_id, train, count=30)

wandb: Agent Starting Run: i1gv5lru with config:
wandb: 	dropout: 0.4227036359887393
wandb: 	last_layer: 64
wandb: 	lr: 0.0031561054119190073
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: svenk. Use `wandb login --relogin` to force relogin


accuracy,▂▄▄█▁▂▆▃█▃
epoch,▁▂▃▃▄▅▆▆▇█
accuracy,0.6243
epoch,10


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 2evqkj70 with config:
wandb: 	dropout: 0.3316228013781351
wandb: 	last_layer: 16
wandb: 	lr: 0.007480029341892443
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


accuracy,▁█
epoch,▁█
accuracy,0.60509
epoch,2


wandb: Agent Starting Run: nggxuyzg with config:
wandb: 	dropout: 0.21460549792446315
wandb: 	last_layer: 16
wandb: 	lr: 0.0038213631980871018
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

wandb: Agent Starting Run: c56nafwo with config:
wandb: 	dropout: 0.2129565585587026
wandb: 	last_layer: 32
wandb: 	lr: 0.0022444649343110355
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


accuracy,▁█
epoch,▁█
accuracy,0.63222
epoch,2


wandb: Agent Starting Run: bgw1cjiz with config:
wandb: 	dropout: 0.440123665847881
wandb: 	last_layer: 64
wandb: 	lr: 0.0002665962475450662
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


accuracy,█▁▂▅
epoch,▁▃▆█
accuracy,0.6152
epoch,4


wandb: Agent Starting Run: 0dw6tsjj with config:
wandb: 	dropout: 0.4365224940932564
wandb: 	last_layer: 64
wandb: 	lr: 0.007901842695940372
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
